In [1]:
import os
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import EOGRegression
import matplotlib 
matplotlib.use('Qt5Agg') 

In [2]:
base_dir = r'C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research'
raw_data_dir = os.path.join(base_dir, 'mne_raw_final')           
filtered_data_dir = os.path.join(base_dir, 'clean_final') 
regression_data_dir = os.path.join(base_dir, 'regression_final')  
epochs_data_dir = os.path.join(base_dir, 'epochs_final')                 
baseline_epochs_data_dir = os.path.join(base_dir, 'baseline_epochs_final')                

On place les différents canaux présent dans notre situation experimentale: 3 canaux EEG, 3 canaux EOG, ainsi que les parametres indiqués dans la description des données ( sfreq et durée d'époch).

In [3]:
# EEG and EOG settings
sampling_freq = 250
channel_names = ['C3', 'Cz', 'C4', 'EOG:ch01', 'EOG:ch02', 'EOG:ch03']
channel_types = ['eeg', 'eeg', 'eeg', 'eog', 'eog', 'eog']
info = mne.create_info(ch_names=channel_names, sfreq=sampling_freq, ch_types=channel_types)
info.set_montage(mne.channels.make_standard_montage('standard_1020'))

event_id_mapping = {'Event_0': 0, 'Event_1': 1}

epoch_time_start, epoch_time_end = -0.2, 4

Combined_data contient tous les fichiers experimentaux bruts avec le type d'epoch associé (1 ou 0). On applique les données défini avant.

In [4]:
#transformation en format mne 
for file in os.listdir(os.path.join(base_dir, 'combined_data')):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(base_dir, 'combined_data', file))
        
        eeg_signals = df[['C3', 'Cz', 'C4']].values.T
        eog_signals = df[['EOG:ch01', 'EOG:ch02', 'EOG:ch03']].values.T
        combined_data = np.vstack([eeg_signals, eog_signals])

        raw = mne.io.RawArray(combined_data, info)
        events = np.array([[i, 0, df['EventType'].iloc[i]] for i in range(len(df)) if df['EventStart'].iloc[i] == 1])
        annotations = mne.Annotations(onset=events[:, 0] / sampling_freq, duration=[4] * len(events),
                                      description=[f"Event_{e}" for e in events[:, 2]], orig_time=None)
        raw.set_annotations(annotations)
        
        raw_file_path = os.path.join(raw_data_dir, f"{os.path.splitext(file)[0]}_raw.fif")
        raw.save(raw_file_path, overwrite=True)
        print(f"Processed and saved {file} to {raw_file_path}")

Creating RawArray with float64 data, n_channels=6, n_times=604803
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Overwriting existing file.
Writing C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0101T_raw.fif
Closing C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0101T_raw.fif
[done]
Processed and saved B0101T.csv to C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0101T_raw.fif
Creating RawArray with float64 data, n_channels=6, n_times=552452
    Range : 0 ... 552451 =      0.000 ...  2209.804 secs
Ready.
Overwriting existing file.
Writing C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0102T_raw.fif
Closing C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0102T_raw.fif
[done]
Processed and saved B0102T.csv to C:\Users\phili\EE

visualisation brut

In [12]:

example_file = r"C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0101T_raw.fif"
raw = mne.io.read_raw_fif(example_file, preload=True)
raw.plot(duration=4, scalings='auto')

Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0101T_raw.fif...
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...


<MNEBrowseFigure size 1916x1032 with 4 Axes>

Channels marked as bad:
none
Channels marked as bad:
none


Les signaux EEG doivent etre taités pour enlever le plus de bruit et d'artefacts possible on va donc les filtrer et utiliser les EOG pour enlever l'activité cerebrale des clignements des yeux:

In [5]:
#filtrage bande passante classique 
for file in os.listdir(raw_data_dir):
    if file.endswith('_raw.fif'):
        raw = mne.io.read_raw_fif(os.path.join(raw_data_dir, file), preload=True)

        raw.filter(l_freq=8., h_freq=30., fir_design='firwin')
        filtered_file_path = os.path.join(filtered_data_dir, file.replace('_raw.fif', 'filtered_raw.fif'))
        raw.save(filtered_file_path, overwrite=True)
        print(f"Saved filtered data: {file} to {filtered_file_path}")


Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\mne_raw_final\B0101T_raw.fif...
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Overwriting existing file.
Writing C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\clean_final\B0101Tfiltered_raw.fif
Closing C:\Users\phili\EEG_proj

In [19]:

example_file = r"C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\clean_final\B0101Tfiltered_raw.fif"
raw = mne.io.read_raw_fif(example_file, preload=True)
raw.plot(duration=4, scalings = 'auto')

Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\clean_final\B0101Tfiltered_raw.fif...
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...


<MNEBrowseFigure size 1916x1032 with 4 Axes>

Pour éliminer les artéfacts liés aux mouvements oculaires (EOG) dans les signaux EEG, on utilise une méthode de régression. Cette technique ajuste les signaux EEG en utilisant les signaux EOG comme régressors pour éliminer les composantes de variance communes. On peut ensuite tester si le regressor de mne à fonctionner en observant la différence de corrélation.

In [6]:

for file in os.listdir(filtered_data_dir):
    if file.endswith('_raw.fif'):
        filtered_file_path = os.path.join(filtered_data_dir, file)

        raw = mne.io.read_raw_fif(filtered_file_path, preload=True)

        raw.set_eeg_reference('average', projection=True)

        eog_regressor = mne.preprocessing.EOGRegression(picks='eeg', picks_artifact='eog')
        eog_regressor.fit(raw)  # Apprentissage du modèle
        raw_cleaned = eog_regressor.apply(raw, copy=True)  # Nettoyage des données

        # Sauvegarde des données nettoyées
        cleaned_file_path = os.path.join(
            regression_data_dir, file.replace('_raw.fif', '_regressed_raw.fif')
        )
        raw_cleaned.save(cleaned_file_path, overwrite=True)
        print(f"Données nettoyées sauvegardées : {file} -> {cleaned_file_path}")


Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\clean_final\B0101Tfiltered_raw.fif...
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Overwriting existing file.
Writing C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\regression_final\B0101Tfiltered_regressed_raw.fif
Closing C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\regression_final\B0101Tfiltered_regressed_raw.fif
[done]
Données nettoyées sauvegardées : B0101Tfiltered_raw.fif -> C:\Users\phili\EEG_project\EEG_pro

verification du bon fonctionnement de la regression : baisse de correlation entre eeg et eog, conservation de tout les échantillions.

In [18]:

example_file = r"C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\regression_final\B0101Tfiltered_regressed_raw.fif"
raw = mne.io.read_raw_fif(example_file, preload=True)
raw.plot(duration=4, scalings = 'auto') 


Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\regression_final\B0101Tfiltered_regressed_raw.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...


<MNEBrowseFigure size 1916x1032 with 4 Axes>

In [44]:
for file in os.listdir(filtered_data_dir):
    if file.endswith('_raw.fif'):
        filtered_file_path = os.path.join(filtered_data_dir, file)
        raw_filtered = mne.io.read_raw_fif(filtered_file_path, preload=True)

        print(f"Canaux dans {file} avant régression :")
        print(raw_filtered.info['ch_names'])  

        # Trouver le fichier nettoyé correspondant
        regressed_file_path = os.path.join(regression_data_dir, file.replace('_raw.fif', '_regressed_raw.fif'))

        if os.path.exists(regressed_file_path):
            raw_regressed = mne.io.read_raw_fif(regressed_file_path, preload=True)

            print(f"Canaux dans {file} après régression :")
            print(raw_regressed.info['ch_names'])  

            eeg_filtered_data = raw_filtered.copy().pick_types(eeg=True).get_data()
            eeg_regressed_data = raw_regressed.copy().pick_types(eeg=True).get_data()

            len_filtered = eeg_filtered_data.shape[1]
            len_regressed = eeg_regressed_data.shape[1]
            length_diff = len_filtered - len_regressed 

            eog_filtered_data = raw_filtered.copy().pick_types(eog=True).get_data()
            eog_regressed_data = raw_regressed.copy().pick_types(eog=True).get_data()

            min_len = min(eeg_filtered_data.shape[1], eeg_regressed_data.shape[1], eog_filtered_data.shape[1])

            eeg_filtered_data = eeg_filtered_data[:, :min_len]
            eeg_regressed_data = eeg_regressed_data[:, :min_len]
            eog_filtered_data = eog_filtered_data[:, :min_len]

            corr_before = np.corrcoef(eeg_filtered_data.flatten(), eog_filtered_data.flatten())[0, 1]

            corr_after = np.corrcoef(eeg_regressed_data.flatten(), eog_regressed_data.flatten())[0, 1]


            print(f"Fichier : {file}")
            print(f"Différence de longueur (avant - après) : {length_diff}")
            print(f"Corrélation EEG-EOG avant nettoyage : {corr_before:.4f}")
            print(f"Corrélation EEG-EOG après nettoyage : {corr_after:.4f}")

Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\clean_final\B0101Tfiltered_raw.fif...
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...
Canaux dans B0101Tfiltered_raw.fif avant régression :
['C3', 'Cz', 'C4', 'EOG:ch01', 'EOG:ch02', 'EOG:ch03']
Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\regression_final\B0101Tfiltered_regressed_raw.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...
Canaux dans B0101Tfiltered_raw.fif après régression :
['C3', 'Cz', 'C4', 'EOG:ch01', 'EOG:ch02', 'EOG:ch03']
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NO

In [7]:
#decoupage en epoch
for file in os.listdir(regression_data_dir):
    if file.endswith('raw.fif'):
        raw = mne.io.read_raw_fif(os.path.join(regression_data_dir, file), preload=True)
        
        try:
            events, _ = mne.events_from_annotations(raw, event_id=event_id_mapping)
        except ValueError as e:
            print(f"Error extracting events for {file}: {e}")
            continue
        
        epochs = mne.Epochs(raw, events, event_id=event_id_mapping, tmin=epoch_time_start, tmax=epoch_time_end,
                            baseline=None, detrend=1, preload=True)
        
        epochs_file_path = os.path.join(epochs_data_dir, file.replace('filtered_regressed_raw.fif', '-epo.fif'))
        epochs.save(epochs_file_path, overwrite=True)
        print(f"Saved epochs for {file} to {epochs_file_path}")



Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\regression_final\B0101Tfiltered_regressed_raw.fif...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Range : 0 ... 604802 =      0.000 ...  2419.208 secs
Ready.
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...
Used Annotations descriptions: ['Event_0', 'Event_1']
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 120 events and 1051 original time points ...
0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
Saved epochs for B0101Tfiltered_regressed_raw.fif to C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\epochs_final\B0101T-epo.fif
Opening raw data file C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\regres

On applique une correction baseline : la baseline est une periode de temps ou le signal et considéré comme stable pour qu'on puisse prendre cette partie de signal en réference. Cela nous permet en théorie d'amplifier l'activité spécifique due à l'evemenent. On prend comme baseline pour chaque epoch la periode 0.2 secondes post evenement, on sait qu'il y à au moins une seconde de repos (stable) après chaque evenement et c'est un standard dans le preprocessing de signal eeg.

In [8]:
baseline_start = -0.2  
baseline_end = 0.0


for file in os.listdir(epochs_data_dir):
    if file.endswith('-epo.fif'):
        epochs = mne.read_epochs(os.path.join(epochs_data_dir, file))

        epochs.apply_baseline(baseline=(baseline_start, baseline_end))

        baseline_epochs_file_path = os.path.join(baseline_epochs_data_dir, file.replace('-epo.fif', '-bcor-epo.fif'))
        epochs.save(baseline_epochs_file_path, overwrite=True)

        print(f"Baseline-corrected and cropped epochs saved: {baseline_epochs_file_path}")


Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\epochs_final\B0101T-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Overwriting existing file.
Baseline-corrected and cropped epochs saved: C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0101T-bcor-epo.fif
Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\epochs_final\B0102T-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =   

In [37]:

# Iterate through each file in the baseline epochs directory
for file in os.listdir(baseline_epochs_data_dir):
    if file.endswith('-epo.fif'):  # Check only .fif files containing epochs
        file_path = os.path.join(baseline_epochs_data_dir, file)
        
        # Read the epochs from the file
        epochs = mne.read_epochs(file_path)
        
        # Check the time points of the first few epochs
        for i, epoch in enumerate(epochs.get_data()):
            # Print out the times for the first few epochs to verify
            print(f"Epoch {i+1} in {file} time range: {epochs.times[0]} to {epochs.times[-1]}")
            print(f"Epoch {i+1} in {file} times: {epochs.times[:10]} ...")  # First 10 time points



Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0101T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Epoch 1 in B0101T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 1 in B0101T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 2 in B0101T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 2 in B0101T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 3 in B0101T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 3 in B0101T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 4 in B0101T-bcor-epo.fif

C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):
C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):


160 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Epoch 1 in B0103T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 1 in B0103T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 2 in B0103T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 2 in B0103T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 3 in B0103T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 3 in B0103T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 4 in B0103T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 4 in B0103T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 5 in B0103T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 5 in B0103T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 6 in B0103T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 6 in B0103T-bco

C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):
C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):


        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =       0.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Epoch 1 in B0202T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 1 in B0202T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 2 in B0202T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 2 in B0202T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 3 in B0202T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 3 in B0202T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 4 in B0202T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 4 in B0202T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 5 in B0202T-bcor-epo.fif time range: 0.0 t

...

C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):
C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):


Epoch 1 in B0803T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 1 in B0803T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 2 in B0803T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 2 in B0803T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 3 in B0803T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 3 in B0803T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 4 in B0803T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 4 in B0803T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 5 in B0803T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 5 in B0803T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 6 in B0803T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 6 in B0803T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 7 in B0803T-bcor-epo.fif time range: 0.0 to 4.

C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):
C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):


160 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Epoch 1 in B0903T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 1 in B0903T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 2 in B0903T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 2 in B0903T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 3 in B0903T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 3 in B0903T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 4 in B0903T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 4 in B0903T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 5 in B0903T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 5 in B0903T-bcor-epo.fif times: [0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036] ...
Epoch 6 in B0903T-bcor-epo.fif time range: 0.0 to 4.0
Epoch 6 in B0903T-bco

C:\Users\phili\AppData\Local\Temp\ipykernel_17548\4206150574.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for i, epoch in enumerate(epochs.get_data()):


In [40]:
example_file = r'C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0101T-bcor-epo.fif'

epochs = mne.read_epochs(example_file)

epochs_info = epochs.info
print("Basic Epochs Info:")
print(epochs_info)
print(f"\nEvent types: {epochs.event_id}")
print(f"\nData shape (n_epochs, n_channels, n_times): {epochs.get_data().shape}")

epochs.plot(picks="all", n_epochs=3, scalings="auto", title="First 3 epochs", show=True)


Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0101T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Basic Epochs Info:
<Info | 11 non-empty values
 bads: []
 ch_names: C3, Cz, C4, EOG:ch01, EOG:ch02, EOG:ch03
 chs: 3 EEG, 3 EOG
 custom_ref_applied: False
 dig: 6 items (3 Cardinal, 3 EEG)
 file_id: 4 items (dict)
 highpass: 8.0 Hz
 lowpass: 30.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 6
 projs: Average EEG reference: on
 sfreq: 250.0 Hz
>

Event types: {'Event_0': 0, 'Event_1': 1}

Data shape (n_epochs, n_channels, n_times): (120, 6, 1051)


C:\Users\phili\AppData\Local\Temp\ipykernel_17548\535137299.py:9: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"\nData shape (n_epochs, n_channels, n_times): {epochs.get_data().shape}")


<MNEBrowseFigure size 1916x1032 with 4 Axes>

Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


On regroupe les epochs pour les feature extractions et le modeling: on creer donc un groupe avec tout les epochs, ceux de l'experience de screening et ceux de l'experience de neurofeedback

In [41]:
all_epochs_list = []

for file in os.listdir(baseline_epochs_data_dir):
    if file.endswith('-epo.fif'):  
        file_path = os.path.join(baseline_epochs_data_dir, file)

        epochs = mne.read_epochs(file_path)

        all_epochs_list.append(epochs)

all_epochs = mne.concatenate_epochs(all_epochs_list)

all_epochs.crop(tmin=0.0, tmax=4.0)

combined_epochs_file_path = os.path.join(baseline_epochs_data_dir, 'all_epochs_combined-epo.fif')

all_epochs.save(combined_epochs_file_path, overwrite=True)

print(f"All epochs have been concatenated, cropped, and saved to {combined_epochs_file_path}")


Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0101T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0102T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading 

C:\Users\phili\AppData\Local\Temp\ipykernel_17548\1197573501.py:16: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_epochs = mne.concatenate_epochs(all_epochs_list)


Not setting metadata
3680 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
All epochs have been concatenated, cropped, and saved to C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\all_epochs_combined-epo.fif


In [9]:
screening_epochs_list = []

for file in os.listdir(baseline_epochs_data_dir):
    # On selectionne tous les fichiers sauf ceux contenant 03T (NF)
    if file.endswith('-epo.fif') and '03T' not in file:
        file_path = os.path.join(baseline_epochs_data_dir, file)
        
        epochs = mne.read_epochs(file_path)
        
        screening_epochs_list.append(epochs)

screening_epochs = mne.concatenate_epochs(screening_epochs_list)

screening_epochs.crop(tmin=0.0, tmax=4.0)

screening_epochs_file_path = os.path.join(baseline_epochs_data_dir, 'screening_epochs_epo.fif')

screening_epochs.save(screening_epochs_file_path, overwrite=True)

print(f"Screened epochs saved to {screening_epochs_file_path}")

Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0101T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0102T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading 

C:\Users\phili\AppData\Local\Temp\ipykernel_17716\3787229874.py:12: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  screening_epochs = mne.concatenate_epochs(screening_epochs_list)


Screened epochs saved to C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\screening_epochs_epo.fif


In [10]:
nf_epochs_list = []

for file in os.listdir(baseline_epochs_data_dir):
    # On selectionne tous les fichiers contenant 03T (NF)
    if file.endswith('-epo.fif') and '03T' in file:
        file_path = os.path.join(baseline_epochs_data_dir, file)
        
        epochs = mne.read_epochs(file_path)
        
        nf_epochs_list.append(epochs)

nf_epochs = mne.concatenate_epochs(nf_epochs_list)

nf_epochs.crop(tmin=0.0, tmax=4.0)

nf_epochs_file_path = os.path.join(baseline_epochs_data_dir, 'nf_epochs_epo.fif')

nf_epochs.save(nf_epochs_file_path, overwrite=True)

print(f"Screened epochs saved to {nf_epochs_file_path}")


Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0103T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0203T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading 

C:\Users\phili\AppData\Local\Temp\ipykernel_17716\1102562227.py:12: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  nf_epochs = mne.concatenate_epochs(nf_epochs_list)


Screened epochs saved to C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\nf_epochs_epo.fif


In [3]:
all_epochs_list = []

for file in os.listdir(baseline_epochs_data_dir):
    if file.endswith('-epo.fif'):  
        file_path = os.path.join(baseline_epochs_data_dir, file)

        epochs = mne.read_epochs(file_path)

        all_epochs_list.append(epochs)

all_epochs = mne.concatenate_epochs(all_epochs_list)

full_epochs_file_path = os.path.join(baseline_epochs_data_dir, 'all_epochs_epo.fif')

all_epochs.save(full_epochs_file_path, overwrite=True)

print(f"All epochs have been concatenated, and saved to {full_epochs_file_path}")

Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0101T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\B0102T-bcor-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 3) active
    Found the data of interest:
        t =    -200.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading 

C:\Users\phili\AppData\Local\Temp\ipykernel_12460\3752200546.py:11: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_epochs = mne.concatenate_epochs(all_epochs_list)


Not setting metadata
3680 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
All epochs have been concatenated, and saved to C:\Users\phili\EEG_project\EEG_proj_data\ucsd-neural-data-challenge\data\research\baseline_epochs_final\all_epochs_epo.fif
